In [ ]:
from google.colab import drive
import os

# Mount Google Drive
drive.mount('/content/drive')

# Create organized folder structure
base_path = "/content/drive/MyDrive/ad_models"
os.makedirs(f"{base_path}/datasets", exist_ok=True)
os.makedirs(f"{base_path}/models", exist_ok=True)
os.makedirs(f"{base_path}/logs", exist_ok=True)

print(f"Folder structure created at: {base_path}")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Folder structure created at: /content/drive/MyDrive/ad_models


In [ ]:
# Basic setup after reset
!pip install torch transformers datasets

!pip install -q accelerate peft bitsandbytes transformers trl
!pip install -q evaluate
!pip install -q peft

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 98.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 75.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 55.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 80.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 38.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
from datasets import load_dataset

ds = load_dataset("smangrul/ad-copy-generation")
print("Train sample 0:", ds["train"][0])  # Check first training example
print("Test sample 0:", ds["test"][0])    # Check first test example

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/714 [00:00<?, ?B/s]

(…)-00000-of-00001-d82840aacfecce9b.parquet:   0%|          | 0.00/166k [00:00<?, ?B/s]

(…)-00000-of-00001-4fa17e6057d2cea3.parquet:   0%|          | 0.00/27.7k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/1000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/141 [00:00<?, ? examples/s]

Train sample 0: {'content': '<s>[INST] <<SYS>>\nCreate a text ad given the following product and description.\n<</SYS>>\n\nProduct: Fitness Magazine\nDescription: Fitness magazine for staying active and achieving your fitness goals. [/INST] Ad: Stay active with a Fitness Magazine! 💪📖 Experience fitness tips and motivating stories. Perfect for fitness enthusiasts and reaching your health and wellness goals. Limited stock - achieve fitness with a touch of motivation! 🌟🌟🏋️\u200d♀️ </s>'}
Test sample 0: {'content': '<s>[INST] <<SYS>>\nCreate a text ad given the following product and description.\n<</SYS>>\n\nProduct: Food Steamer\nDescription: Steam your way to healthier meals with a Food Steamer - designed for preserving nutrients and flavors. [/INST] Ad: Steam, nourish, and embrace healthy eating! 🥦🌟 Cook with the power of steam. Perfect for health-conscious cooks and retaining the goodness of your dishes with a touch of food steamer vitality! 🌟🍳🌿 </s>'}


In [ ]:
from datasets import load_dataset

# Load the dataset
dataset = load_dataset("smangrul/ad-copy-generation", split="train")

def format_prompts(examples):
    import re
    texts = []
    for sample in examples["content"]:
        match = re.search(r'Product: (.*?)\nDescription: (.*?)\[/INST\] Ad: (.*?) </s>', sample)
        if match:
            product, description, ad = match.groups()
            texts.append(
                f"<s>[INST] <<SYS>>\nCreate a text ad.\n<</SYS>>\n\nProduct: {product.strip()}\nDescription: {description.strip()} [/INST] Ad: {ad.strip()} </s>"
            )
        else:
            texts.append(sample)  # fallback
    return {"text": texts}

# Appliquez la mise en forme au dataset
dataset = dataset.map(format_prompts, batched=True)

# Example after formatting
print(dataset[0])  # Check first entry to ensure it looks like: {"text": "formatted prompt"}


Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

{'content': '<s>[INST] <<SYS>>\nCreate a text ad given the following product and description.\n<</SYS>>\n\nProduct: Fitness Magazine\nDescription: Fitness magazine for staying active and achieving your fitness goals. [/INST] Ad: Stay active with a Fitness Magazine! 💪📖 Experience fitness tips and motivating stories. Perfect for fitness enthusiasts and reaching your health and wellness goals. Limited stock - achieve fitness with a touch of motivation! 🌟🌟🏋️\u200d♀️ </s>', 'text': '<s>[INST] <<SYS>>\nCreate a text ad.\n<</SYS>>\n\nProduct: Fitness Magazine\nDescription: Fitness magazine for staying active and achieving your fitness goals. [/INST] Ad: Stay active with a Fitness Magazine! 💪📖 Experience fitness tips and motivating stories. Perfect for fitness enthusiasts and reaching your health and wellness goals. Limited stock - achieve fitness with a touch of motivation! 🌟🌟🏋️\u200d♀️ </s>'}


In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
import torch
from peft import prepare_model_for_kbit_training

from huggingface_hub import login

# Replace 'your_token_here' with your actual Hugging Face token
login(token="")



# Define model ID
model_id = "mistralai/Mistral-7B-Instruct-v0.3"

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_id)

# Define the Bits and Bytes configuration for 4-bit quantization
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

# Load the model with the quantization config
model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config)

# Enable gradient checkpointing to save memory
model.gradient_checkpointing_enable()

# Prepare the model for k-bit training (to handle quantized model)
model = prepare_model_for_kbit_training(model)

print(f"Model loaded: {model.device}")


from inspect import signature
print(signature(model.forward))

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Model loaded: cuda:0
(input_ids: Optional[torch.LongTensor] = None, attention_mask: Optional[torch.Tensor] = None, position_ids: Optional[torch.LongTensor] = None, past_key_values: Optional[transformers.cache_utils.Cache] = None, inputs_embeds: Optional[torch.FloatTensor] = None, labels: Optional[torch.LongTensor] = None, use_cache: Optional[bool] = None, output_attentions: Optional[bool] = None, output_hidden_states: Optional[bool] = None, cache_position: Optional[torch.LongTensor] = None, logits_to_keep: Union[int, torch.Tensor] = 0, **kwargs: *<class 'transformers.models.mistral.modeling_mistral.KwargsForCausalLM'>) -> transformers.modeling_outputs.CausalLMOutputWithPast


In [ ]:
!pip list | grep -E "transformers|trl|peft|accelerate"

accelerate                            1.5.2
fastrlock                             0.8.3
peft                                  0.14.0
sentence-transformers                 3.4.1
transformers                          4.51.1
trl                                   0.16.1


In [ ]:
from peft import LoraConfig, get_peft_model, TaskType

# Define LoRA configuration
config = LoraConfig(
    r=32,                         # LoRA rank (adjust as needed)
    lora_alpha=64,                # Scaling factor
    target_modules=[
        "q_proj", "k_proj", "v_proj", "o_proj",  # Attention layers
        "gate_proj", "up_proj", "down_proj", "lm_head"  # Additional layers
    ],
    bias="none",                  # No bias for LoRA layers
    lora_dropout=0.05,            # Dropout for LoRA layers
    task_type=TaskType.CAUSAL_LM, # Task type: Causal Language Modeling
)

# Get the PEFT model
model = get_peft_model(model, config)

In [ ]:
from transformers import TrainingArguments

args = TrainingArguments(
    output_dir=f"{base_path}/models/ad_mistral_lora",
    per_device_train_batch_size=4,
    gradient_accumulation_steps=4,  # effective batch size = 16
    num_train_epochs=3,
    learning_rate=2e-5,
    fp16=True,  # ✅ Important for Colab memory and speed
    logging_dir=f"{base_path}/logs",
    save_strategy="epoch",
    logging_steps=10,
    save_total_limit=2,
    report_to="none",  # Disable W&B logging
)

In [ ]:
from transformers import DataCollatorForLanguageModeling
from trl import SFTTrainer

# 1. Tokenize the dataset
def tokenize_function(examples):
    return tokenizer(examples['text'], truncation=True, padding="max_length", max_length=256)

tokenized_dataset = dataset.map(tokenize_function, batched=True)

# 2. Ensure the dataset has a 'labels' column (set labels equal to input_ids for causal LM)
if "labels" not in tokenized_dataset.column_names:
    tokenized_dataset = tokenized_dataset.map(lambda x: {"labels": x["input_ids"]})

# 3. Use a data collator for causal language modeling
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False  # Causal LM does not use masked language modeling
)

# 4. Define the trainer
trainer = SFTTrainer(
    model=model,               # The model with LoRA applied
    train_dataset=tokenized_dataset,  # The tokenized dataset you prepared
    args=args,                 # Training arguments
    data_collator=data_collator  # Pass the correct data collator
)

# 5. Start training
trainer.train()


Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Truncating train dataset:   0%|          | 0/1000 [00:00<?, ? examples/s]

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Step,Training Loss
10,2.374500
20,1.443400
30,1.249700
40,1.132600
50,1.021800
60,0.965100
70,0.826400
80,0.717800
90,0.697700
100,0.694700


/usr/local/lib/python3.11/dist-packages/peft/utils/save_and_load.py:230: UserWarning: Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.
  warnings.warn("Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.")
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.11/dist-packages/peft/utils/save_and_load.py:230: UserWarning: Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.
  warnings.warn("Setting `save_embedding_layers` to `True` as embeddi

TrainOutput(global_step=186, training_loss=0.8604911129961732, metrics={'train_runtime': 3290.5412, 'train_samples_per_second': 0.912, 'train_steps_per_second': 0.057, 'total_flos': 3.273010144346112e+16, 'train_loss': 0.8604911129961732})

In [ ]:
# Merge the model with the adapter
adapter_model = trainer.model
merged_model = adapter_model.merge_and_unload()

# Get the trained tokenizer
trained_tokenizer = trainer.tokenizer


Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.


In [ ]:
# Define your output path
output_dir = f"{base_path}/models/mistral-7b-instruct-v0.3-finetuned"

# Save the merged model
merged_model.save_pretrained(output_dir)

# Save the tokenizer
tokenizer.save_pretrained(output_dir)

print(f"✅ Model and tokenizer saved to: {output_dir}")


✅ Model and tokenizer saved to: /content/drive/MyDrive/ad_models/models/mistral-7b-instruct-v0.3-finetuned


/usr/local/lib/python3.11/dist-packages/peft/utils/save_and_load.py:230: UserWarning: Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.
  warnings.warn("Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.")
